<h1>VacationPy: An exercise in making API calls to analyze data.</h1>

This code will use Geocoding and API calls to OpenWeather to analyze data from 500 Worldwide cities in order to identify an ideal vacation spot in terms of pleasant weather.

In [3]:
# Dependencies
import requests
import json
from citipy import citipy
import numpy as np
from config import owkey
import geoip2.database
import geoip2.webservice
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
import gmaps
from config import gkey
import pprint
import time

In [6]:
#Import CSV file with city data
file="worldcitiespop_reduced.csv"
cities_df=pd.read_csv(file)

In [7]:
#Extract a random sample (without replacement) of 500 cities from the CSV file
simpcities_df = cities_df[["Latitude","Longitude"]]
cities_dict=simpcities_df.set_index("Latitude")["Longitude"].to_dict()
random_500=dict(random.sample(cities_dict.items(), 500))

In [8]:
#Create a dataframe with weather info

weather_df= pd.DataFrame({
    "Name": [], 
    "Temperature (F)": [],
    "Max Temperature (F)":[],
    "Humidity (%)": [], 
    "Cloudiness (%)": [], 
    "Windspeed (mph)": [], 
    
})

#Loop through the dataframe to make API calls and get the weather in 500 cities

call_counter=0
for lat in random_500:
    api_call = "http://api.openweathermap.org/data/2.5/weather?lat=" + str(lat) + "&lon=" + str(random_500[lat]) + "&units=imperial" + "&appid=" + owkey
    api_response = requests.get(api_call)
    api_json = api_response.json()
    name=api_json["name"]
    temp=api_json["main"]["temp"]
    temp_max=api_json["main"]["temp_max"]
    humidity=api_json["main"]["humidity"]
    clouds=api_json["clouds"]["all"]
    speed=api_json["wind"]["speed"]
    
    #append the weather values to the dataframe
    weather_df=weather_df.append({'Name' : name , 
                                  'Temperature (F)' : temp, 
                                  "Humidity (%)": humidity, 
                                  "Cloudiness (%)": clouds, 
                                  "Windspeed (mph)": speed,
                                  "Max Temperature (F)":temp_max,
                                  "Latitude": lat, 
                                  "Longitude": random_500[lat] } , ignore_index=True)
    
    call_counter+=1
    if call_counter==19:
        time.sleep(1)
        call_counter=0
        
        continue

In [11]:
weather_df["Distance From Equator"]=abs(weather_df["Latitude"])
weather_df

,Name,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Windspeed (mph),Latitude,Longitude,Distance From Equator
0,Elva linn,62.98,64.40,72.0,1.0,13.87,58.228333,26.377500,58.228333
1,Mirande,62.44,63.00,78.0,100.0,1.99,43.537521,0.499651,43.537521
2,Arafo,69.35,71.01,43.0,43.0,5.01,28.339708,-16.422435,28.339708
3,Puerto López,78.37,78.37,81.0,83.0,6.73,4.110444,-72.858903,4.110444
4,Égriselles-le-Bocage,60.01,60.01,64.0,38.0,4.72,48.102000,3.162920,48.102000
...,...,...,...,...,...,...,...,...,...
495,Saurimo,71.06,71.06,37.0,87.0,5.77,-9.357450,20.482987,9.357450
496,Kafr ash Shaykh,70.36,70.36,86.0,0.0,2.53,31.193611,30.992222,31.193611
497,Tielou,51.73,51.73,77.0,97.0,3.18,32.862641,104.187656,32.862641
498,Nousiainen,57.42,59.00,87.0,40.0,5.82,60.641111,22.149167,60.641111


In [17]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)
weather_df = weather_df.dropna()
humidity_weight = weather_df["Humidity (%)"].astype(float)

In [18]:

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig 

Figure(layout=FigureLayout(height='420px'))

In [14]:
#Filter the data to find the locatons with the best weather

#----Filter by temperature
newlocs_df=weather_df.loc[weather_df["Max Temperature (F)"]>70]
newlocs_df=newlocs_df.loc[newlocs_df["Max Temperature (F)"]<80]

#----Filter by windspeed
newlocs_df=newlocs_df.loc[newlocs_df["Windspeed (mph)"]<10]


#----Filter by cloudiness
newlocs_df=newlocs_df.loc[newlocs_df["Cloudiness (%)"]==0]


In [15]:
#Build new heat map of the (reduced) list of ideal weather locations

newlocs = newlocs_df[["Latitude", "Longitude"]].astype(float)
humidity_weight2 = newlocs_df["Humidity (%)"].astype(float)

fig2 = gmaps.figure()
heat_layer2 = gmaps.heatmap_layer(newlocs, weights=humidity_weight2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig2.add_layer(heat_layer2)

fig2

Figure(layout=FigureLayout(height='420px'))

In [24]:
#Use Geocode API to loop through the list of ideal weather locations and get
#info about the nearest hotel within 50,000 meters of each (the homework said 5,000, but this yielded few results).

##NOTE: because data is sampled randomly and Geocode data is incomplete, sometimes missing values are returned---

#----Create new columns in the dataframe to hold new info

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
newlocs_df["City"] = ""
newlocs_df["Country"]=""
newlocs_df["City"]=""
#newlocsindex=newlocs_df.setindex("Name")


#----Get the nearest hotel by latitude and longitude
for index, row in newlocs_df.iterrows():
    
    hotelzone_lat = row['Latitude']
    hotelzone_long=row['Longitude']
    
    params = {
    "location": str(hotelzone_lat)+","+str(hotelzone_long),
    "radius": 50000,
    "input": "",
    "type":"lodging",
    "key": gkey}


    
    response = requests.get(base_url, params=params).json()
    results = response['results']

#----Try to print the hotel name, also save that to the dataframe

    try:
        print(f"Hotel name is {results[0]['name']}.")
        
        newlocs_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Hotel name not found.")
        
#----Make an new API call to collect country and city     
    newbase_url="https://maps.googleapis.com/maps/api/geocode/json?"
    
    params = {
    "latlng": str(hotelzone_lat)+","+str(hotelzone_long),
    "key": gkey}
    

#----Try to print the hotel city and country, also save that to the dataframe     
    try:
        response2 = requests.get(newbase_url, params=params).json()
        city= newlocs_df.loc[index, 'City']=response2["results"][0]["address_components"][2]["long_name"]
        newlocs_df.loc[index, 'City'] = city
        print(f"City is {city}.")
    except:
        print("City not found.")
    try:
        country= newlocs_df.loc[index, 'Country']=response2["results"][0]["address_components"][5]["long_name"]
        newlocs_df.loc[index, 'Country'] = country
        print(f"Country is {country}.")
        
    except (KeyError, IndexError):
        print("Country not found.")
        
    print("------------")
    

Hotel name is AHSAN HOUSE , AHSAN Calony.
City is Kandahar.
Country not found.
------------
Hotel name is Tirana International Hotel.
City is Qarku i Tiranës.
Country not found.
------------
Hotel name is Parador de El Saler.
City is Almoines.
Country is Spain.
------------
Hotel name is محل اقامت عبدالصبور زړور.
City is Baghlan.
Country not found.
------------
Hotel name is Base vida Belarusangola.
City is Malanje Province.
Country not found.
------------
Hotel name is فدامحمد.
City is Afghanistan.
Country not found.
------------
Hotel name not found.
City is Afghanistan.
Country not found.
------------
Hotel name is Ashab Baba Ghar.
City is Wardak.
Country not found.
------------
Hotel name is Post And Telecommunication Hotel.
City is Hami Shi.
Country not found.
------------
Hotel name is Hotel Atina.
City is Bosnia and Herzegovina.
Country not found.
------------
Hotel name is مسجد ترگمنها سرآسیاب کندز.
City is Kunduz.
Country not found.
------------
Hotel name not found.
City is A

In [22]:
#Create a column in the dataframe that combines the relevant info so that I can drop it in the infobox

newlocs_df["BoxStuff"]= (newlocs_df["Hotel Name"] + ": "+ newlocs_df["City"]) + ", " +newlocs_df["Country"]

In [12]:
#Make those infoboxes

boxes = newlocs_df["BoxStuff"].tolist()

# Create a map layer that flags the hotels and puts my infoboxes inside them

last_locs = newlocs_df[["Latitude", "Longitude"]].astype(float)

hotel_layer = gmaps.symbol_layer(
    last_locs, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[item for item in boxes])

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
results

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 31.037634, 'lng': 31.355686},
   'viewport': {'northeast': {'lat': 31.0390190802915,
     'lng': 31.35707233029149},
    'southwest': {'lat': 31.0363211197085, 'lng': 31.35437436970849}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': 'e988eff0dce4e73f1d0ee13777d66f7adbe3b3ea',
  'name': 'Almadiafah',
  'photos': [{'height': 3264,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116583827730062353328">Marwan Medhat</a>'],
    'photo_reference': 'CmRaAAAAkBkSMTju6TSEBJd4R_kqSKd46jmnqV55eyPqJ1o_fA0SVxrBLek6xCWiNQVqsyir2ANoO70sLBwwDFQLVwKHCkn_ieweXEied86ZEQL5-OuQdDuGoVBKzuSYTsiKzhMPEhB3H4ywJyG2Yscfp5PzuK1wGhTBRKb4K0_DUG6kbNEHbtosbve9Ag',
    'width': 2448}],
  'place_id': 'ChIJcXHdNdWd9xQRP5z3LA6x0Yc',
  'plus_code': {'compound_code': '29Q4+37 Mansoura, Mansoura Qism 2, Mansoura, Egypt',
   'global_code': '8G3H29Q4+37'},
  'rating': 3.7,
  'reference': 'Ch